This function computes the gradient of the flow map $ \nabla \mathbf{F}_{t_0}^{t}(\mathbf{x_0}) $ at $ \mathbf{x}_0 $ at time $ t $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x}_0 $ |
| X | array (Ny, Nx) | X-meshgrid of velocity data|
| Y | array (Ny, Nx) | Y-meshgrid of velocity data|
| Interpolant | list (2,) | Interpolant[0]: Interpolant object for $ u((\mathbf{x}, t))  $ <br /> Interpolant[1]: Interpolant object for $ v((\mathbf{x}, t))  $. <br /> <br /> The interpolation, however, works only <br /> if the particle is in the defined flow domain. <br /> At the boundary of the domain, <br /> the interpolant is not defined.|
| periodic | list (2,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br />|
| defined_domain | array (Ny, Nx) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| time_data | array(1,Nt) | time of velocity data |
| delta | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | list (2,) | aux_grid[0]: dx auxiliary spacing <br /> aux_grid[1]: dy auxiliary spacing |
| gradFmap | array (2,2) | $ \nabla \mathbf{F}_{t_0}^{t}(\mathbf{x_0}) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path in order to access the functions
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy
import numpy as np

# Import package which checks particle location
from ipynb.fs.defs.check_location import check_location

# Import package for progress bar
from tqdm.notebook import tqdm

# function to compute trajectory from dFdt
from ipynb.fs.defs.integration_dFdt import integration_dFdt

import matplotlib.pyplot as plt

In [3]:
def gradient_flowmap(time, x, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid):
    
    # define auxiliary grid spacing
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    x = x.reshape(2, -1)
    
    X0, XL, XR, XU, XD = [], [], [], [], []

    nan_mask = []
    
    for i in range(x.shape[1]):
        
        xr = x[0, i] + rho_x
        xl = x[0, i] - rho_x
        yu = x[1, i] + rho_y
        yd = x[1, i] - rho_y
        
        bool_xr = (check_location(X, Y, defined_domain, np.array([xr, x[1, i]]))[0] == "IN")
        bool_xl = (check_location(X, Y, defined_domain, np.array([xl, x[1, i]]))[0] == "IN")
        bool_yu = (check_location(X, Y, defined_domain, np.array([x[0, i], yu]))[0] == "IN")
        bool_yd = (check_location(X, Y, defined_domain, np.array([x[0, i], yd]))[0] == "IN")
        
        # check initial location of particles. Only compute gradient of flow map for those particles whose auxiliary trajectories are all within the defined flow domain
        if bool_xr and bool_xl and bool_yu and bool_yd:
            
            nan_mask.append(True)
            
        else:
            
            nan_mask.append(False)
    
        X0.append([x[0, i], x[1, i]])
        XL.append([xl, x[1, i]])
        XR.append([xr, x[1, i]])
        XU.append([x[0, i], yu])
        XD.append([x[0, i], yd])
    
    X0 = np.array(X0).transpose()
    XL = np.array(XL).transpose()
    XR = np.array(XR).transpose()
    XU = np.array(XU).transpose()
    XD = np.array(XD).transpose()
    
    # launch trajectories from auxiliary grid
    XLend = integration_dFdt(time, XL, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)[0]
    XRend = integration_dFdt(time, XR, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)[0]
    XDend = integration_dFdt(time, XD, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)[0]
    XUend = integration_dFdt(time, XU, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)[0]
    gradFmap = np.zeros((XLend.shape[0], 2, 2, XLend.shape[2]))
    
    # compute gradient of flow map over time interval
    
    for i in range(X0.shape[1]):
        
        if nan_mask[i]:
        
            for j in range(len(time)):

                gradFmap[j,0,0,i] = (XRend[j,0,i]-XLend[j,0,i])/(2*rho_x)
                gradFmap[j,1,0,i] = (XRend[j,1,i]-XLend[j,1,i])/(2*rho_x)
        
                gradFmap[j,0,1,i] = (XUend[j,0,i]-XDend[j,0,i])/(2*rho_y)
                gradFmap[j,1,1,i] = (XUend[j,1,i]-XDend[j,1,i])/(2*rho_y)
                
        else:
            
            gradFmap[:,:,:,i] = np.nan
    
    print("Computation of gradient of flow map: DONE")
    
    return gradFmap